In [ ]:
import numpy as np

class BinaryLogisticRegression:
    def __init__(self, n_features, batch_size, conv_threshold):
        self.n_features = n_features
        self.weights = np.zeros(n_features + 1)  # extra element for bias
        self.alpha = 0.01
        self.batch_size = batch_size
        self.conv_threshold = conv_threshold

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def train(self, X, Y):
        # intializing values
        converge = False
        epochs = 0
        n_examples = X.shape[0]

        while not converge:
            # update # of epochs
            epochs += 1
            # acquire indices for shuffling of X and Y
            indices = np.arange(n_examples)
            np.random.shuffle(indices)
            X = X[indices]
            Y = Y[indices]
            # calc last epoch loss
            last_epoch_loss = self.loss(X, Y)
            # for the # of batches
            for i in range(0, n_examples, self.batch_size):
                X_batch = X[i:i + self.batch_size]
                Y_batch = Y[i:i + self.batch_size]
                # reinitialize gradient to be 0s
                grad = np.zeros(self.weights.shape)
                # for each pair in the batch
                for x, y in zip(X_batch, Y_batch):
                    prediction = self.sigmoid(np.dot(self.weights, x))
                    # gradient calculation
                    error = prediction - y
                    grad += error * x
                # update weights
                self.weights -= (self.alpha * grad) / self.batch_size
            epoch_loss = self.loss(X, Y)
            if abs(epoch_loss - last_epoch_loss) < self.conv_threshold:
                converge = True
        return epochs

    def loss(self, X, Y):
        n_examples = X.shape[0]
        total_loss = 0
    
        for i in range(n_examples):
            linear_output = np.dot(self.weights, X[i])
            y = 1 if Y[i] == 1 else -1  # Convert labels to {-1, 1}
            # compute logistic loss
            logistic_loss = np.log(1 + np.exp(-y * linear_output))
            total_loss += logistic_loss
    
        return total_loss / n_examples

    
    def predict(self, X):
        # multiply X by weights of model
        predictions = self.sigmoid(X @ self.weights)
        return np.where(predictions >= 0.5, 1, 0)

    def accuracy(self, X, Y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == Y)
        return accuracy
